In [28]:
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
from enum import Enum
import re

In [4]:
init_url = "https://www.nature.com/articles/s41560-024-01529-3"

In [89]:
dois = []
docs = []

In [90]:
def get_text(soup):
    for script in soup(['script', 'style']):
        script.extract()
    text = soup.get_text(separator=' ')
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text

In [91]:
def get_dois(url, max_depth, curr_depth=1):
    if curr_depth == max_depth:
        return []
    page = urlopen(url)
    html_content = page.read().decode("utf-8")
    soup = BeautifulSoup(html_content, "html.parser")
    a_tags = soup.find_all('a', href=True)
    doi_pattern = r'https:\/\/doi.org\/10.+'
    for a in a_tags:
        match = re.search(doi_pattern, a['href'], re.IGNORECASE)
        if match:
            new_url = match.group()
            if new_url not in dois:
                dois.append(new_url)
                docs.append(get_text(soup))
                get_dois(new_url, max_depth, curr_depth+1)
    return dois

In [92]:
get_dois(init_url, 20)

HTTPError: HTTP Error 403: Forbidden

In [84]:
print(dois)

['https://doi.org/10.1038/s41560-024-01623-6', 'https://doi.org/10.1038/s41560-024-01529-3', 'https://doi.org/10.1038/s41560-024-01623-6', 'https://doi.org/10.1038/s41560-024-01529-3', 'https://doi.org/10.1038/s41560-024-01623-6', 'https://doi.org/10.1038/s41560-024-01529-3', 'https://doi.org/10.1038/s41560-024-01623-6', 'https://doi.org/10.1038/s41560-024-01529-3', 'https://doi.org/10.1038/s41560-024-01623-6', 'https://doi.org/10.1038/s41560-024-01529-3', 'https://doi.org/10.1038/s41560-024-01623-6', 'https://doi.org/10.1038/s41560-024-01529-3', 'https://doi.org/10.1038/s41560-024-01623-6', 'https://doi.org/10.1038/s41560-024-01529-3', 'https://doi.org/10.1038/s41560-024-01623-6', 'https://doi.org/10.1038/s41560-024-01529-3', 'https://doi.org/10.1038/s41560-024-01623-6', 'https://doi.org/10.1038/s41560-024-01529-3', 'https://doi.org/10.1038/s41560-024-01623-6', 'https://doi.org/10.1126%2Fscience.adm9474', 'https://doi.org/10.1038%2Fs41566-019-0398-2', 'https://doi.org/10.1038%2Fs41560

In [93]:
print(docs)

['Molecular cation and low-dimensional perovskite surface passivation in perovskite solar cells | Nature Energy\nSkip to main content\nThank you for visiting nature.com. You are using a browser version with limited support for CSS. To obtain\nthe best experience, we recommend you use a more up to date browser (or turn off compatibility mode in\nInternet Explorer). In the meantime, to ensure continued support, we are displaying the site without styles\nand JavaScript.\nAdvertisement\nView all journals\nSearch\nLog in\nExplore\ncontent\nAbout\nthe journal\nPublish\nwith us\nSubscribe\nSign up for alerts\nRSS feed\nnature\nnature energy\nreview articles\narticle\nReview Article\nPublished:\n04 July 2024\nMolecular cation and low-dimensional perovskite surface passivation in perovskite solar cells\nSam Teale\nORCID:\norcid.org/0000-0001-9638-3453 1 \xa0 na1 \xa0 nAff4 ,\nMatteo Degani\nORCID:\norcid.org/0000-0001-7422-331X 2 \xa0 na1 ,\nBin Chen\nORCID:\norcid.org/0000-0002-2106-7664 1 , 3

In [94]:
df = pd.DataFrame({
    'id': [i for i in range(1, len(dois) + 1)],
    'url': dois,
    'text': docs
})
df.head()

,id,url,text
0,1,https://doi.org/10.1038/s41560-024-01623-6,Molecular cation and low-dimensional perovskit...
1,2,https://doi.org/10.1038/s41560-024-01529-3,Author Correction: Molecular cation and low-di...
2,3,https://doi.org/10.1126%2Fscience.adm9474,Molecular cation and low-dimensional perovskit...
